In [4]:
# 新しく書き直し

# raw fluor, neuropil, deconvの波形からneuronを抽出
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import tifffile
import cv2
import random
# from setGUIFont import *
import datetime
import time

from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QColor, QFont, QPainterPath, QBrush
from PyQt5.QtCore import Qt, QTimer, QItemSelection
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

dir_notebook = os.path.dirname(os.path.abspath("__file__"))
# 親ディレクトリのパスを取得
dir_parent = os.path.dirname(dir_notebook)
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

from optic.config import *
from optic.gui import *
from optic.io import *
from optic.utils import *
from optic.preprocessing import *

# アプリ名の設定
AppSettings.setCurrentApp("SUITE2P_ROI_CHECK")

class Suite2pROICheckGUI(QMainWindow, WidgetManager):
    def __init__(self):
        """
        pri: 格納するkey
        """
        QMainWindow.__init__(self)
        WidgetManager.__init__(self)
        # self.class_HoloStimResponse = None

        # Tableの列名と列番号, 列情報のdict
        self.dict_tablecol = AppSettings.getTableColumns()
        # キーと機能のマッピング
        self.key_function_map = AppSettings.getKeyFunctionMap()
        # GUIのコンフィグ
        self.gui_defaults = AppSettings.getGuiDefaults()

        self.setWindowTitle(self.gui_defaults["TITLE"])
        self.setGeometry(self.gui_defaults["INIT_POSITION_X"], self.gui_defaults["INIT_POSITION_Y"], self.gui_defaults["WIDTH"], self.gui_defaults["HEIGHT"])
        self.setupUI_done = False
        
        # self.initializeAttributes()

        self.setupFileLoadUI()

    """
    setup UI Function
    """
    def setupFileLoadUI(self):
        self.makeLayoutMain()
        self.addLayoutFileLoadUI()

    """
    makeLayout Function
    """
    # Main Layout
    def makeLayoutMain(self):
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        self.mainLayout = QGridLayout(central_widget)

    # ファイル読み込み用UI Layout
    def makeLayoutFileLoadUI(self):
        layout = QVBoxLayout()
        app_keys = self.gui_defaults["APP_KEYS"]
        # LineEdit
        list_label = ["Fall mat file path", "Reference Tiff image file path (optional)", "Cellpose Mask path (optional)"]
        list_key = [f"path_fall_{app_keys}", f"path_reftif_{app_keys}", f"path_cellpose_{app_keys}"]
        list_filetype = ["mat", "tiff", "npy"]
        for label, key, filetype in zip(list_label, list_key, list_filetype):
            layout.addLayout(makeLayoutLoadFileWidget(self, label=label, key=key, filetype=filetype))
        # Button
        layout_button = QHBoxLayout()
        list_label = ["Load Files", "Exit", "Help"]
        list_key = ["load_file", "exit", "help"]
        list_func_ = [None, lambda: exitApp(self), None]
        for label, key, func_ in zip(list_label, list_key, list_func_):
            layout_button.addWidget(self.makeWidgetButton(key=key, label=label, func_=func_))
        layout.addLayout(layout_button)
        return layout

    def addLayoutFileLoadUI(self):
        self.mainLayout.addLayout(self.makeLayoutFileLoadUI(), 0, 0, 1, 1)

if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
#     font = setGUIFont(app)
    gui = Suite2pROICheckGUI()
    gui.show()
    sys.exit(app.exec_())

SystemExit: 0

In [4]:
?gui.makeWidgetButton

Signature: gui.makeWidgetButton(key, label, func_=None)
Docstring: <no docstring>
File:      d:\optic\optic\gui\widget_manager.py
Type:      method

In [7]:
gui.dict_button

{'path_fall_pri': <PyQt5.QtWidgets.QPushButton at 0x132a7b42160>,
 'path_reftif_pri': <PyQt5.QtWidgets.QPushButton at 0x132a7b423a0>,
 'path_cellpose_pri': <PyQt5.QtWidgets.QPushButton at 0x132a7b42670>,
 'load_file': <PyQt5.QtWidgets.QPushButton at 0x132a7b42700>,
 'exit': <PyQt5.QtWidgets.QPushButton at 0x132a7b42790>,
 'help': <PyQt5.QtWidgets.QPushButton at 0x132a7b42820>}